<a href="https://colab.research.google.com/github/dhenifferraujo/Machine-Learning/blob/main/Ex04_GBM_XGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SCC-ICMC-USP - 1o. semestre de 2021
# SCC5871/MAI5025 - APRENDIZADO DE MÁQUINA
# **Exercício 4**

### Profa. Roseli A. F. Romero

### PAE: Guilherme V. Nardari



Considere o arquivo `houses_to_rent_v2.csv`. Esta base tem dados de imóveis para 5 cidades do Brasil. Esta base pode ser usada tanto pelo proprietário quanto pelo locatário. Pretende-se que um proprietario
tenha uma estimativa do preço que pode alugar seu imóvel, através de seus atributos. Por outro lado, pretende-se que um locatário possa estimar se o aluguel é condizente.

### Questão 1 (Mesmo do ex. anterior)
 
 - Separe o conjunto em treinamento (80%) e teste (20%)
 - Verifique se é necessário um preprocessamento nos dados. Justifique o(s) tipo (s) de preprocessamento que foram feitos
 - Dica: O Sklearn não trabalha com atributos nominais em árvores de decisão

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from math import sqrt
import pandas as pd
import numpy as np

data = pd.read_csv('houses_to_rent_v2.csv', na_values='-')
# Vamos ignorar algumas colunas para simplificar o problema
data = data[['city', 'rooms', 'bathroom', 'parking spaces', 'furniture', 'fire insurance (R$)', 'rent amount (R$)']]
data = data.reset_index(drop=True)
print(data.shape[0])
print(data.info())

10692
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10692 entries, 0 to 10691
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   city                 10692 non-null  object
 1   rooms                10692 non-null  int64 
 2   bathroom             10692 non-null  int64 
 3   parking spaces       10692 non-null  int64 
 4   furniture            10692 non-null  object
 5   fire insurance (R$)  10692 non-null  int64 
 6   rent amount (R$)     10692 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 584.8+ KB
None


In [ ]:
labels_city = set(data['city'])
labels_furniture = set(data['furniture'].head())

data.head()

,city,rooms,bathroom,parking spaces,furniture,fire insurance (R$),rent amount (R$)
0,São Paulo,2,1,1,furnished,42,3300
1,São Paulo,4,4,0,not furnished,63,4960
2,Porto Alegre,1,1,1,not furnished,41,2800
3,Porto Alegre,2,1,0,not furnished,17,1112
4,São Paulo,1,1,0,not furnished,11,800


In [ ]:
#Estamos utilizando o LabelEncoder do sklearn para converter os valores categóricos em valores numéricos

label_encoder1 = LabelEncoder()
data['furniture'] = label_encoder1.fit_transform(data['furniture'])

label_encoder2 = LabelEncoder()
data['city'] = label_encoder2.fit_transform(data['city'])

data.head()

,city,rooms,bathroom,parking spaces,furniture,fire insurance (R$),rent amount (R$)
0,4,2,1,1,0,42,3300
1,4,4,4,0,1,63,4960
2,2,1,1,1,1,41,2800
3,2,2,1,0,1,17,1112
4,4,1,1,0,1,11,800


In [ ]:
print(str(label_encoder1.classes_) + ' -> '  + str(set(data['furniture'].head())))
print(str(label_encoder2.classes_) + ' -> '  + str(set(data['city'])))

['furnished' 'not furnished'] -> {0, 1}
['Belo Horizonte' 'Campinas' 'Porto Alegre' 'Rio de Janeiro' 'São Paulo'] -> {0, 1, 2, 3, 4}


In [ ]:
#Aqui nós estamos separando o target do dataset e separando o conjunto em treinamento (80%) e teste (20%)
x = data.iloc[:,:-1]
y = data.iloc[:,-1:]

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=0)

In [ ]:
y_train.values.reshape(-1)

array([1450, 1580, 3400, ..., 4000, 4500, 1300])

### Questão 2

Treine um modelo de regressão com o método de Gradient Boosting no conjunto de treinamento. Calcule o Mean Squared Error (MSE) e o R2 no conjunto de teste.

Explore ao menos 5 configurações de hiperparâmetros, variando 
- Profundidade máxima dos estimadores
- Nro de estimadores
- Taxa de aprendizado
- A fração de exemplos para treinamento dos estimadores
- Dica: Utilize `sklearn.ensemble.GradientBoostingRegressor`

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth':[4], 
              'random_state':[2], 
              'learning_rate':[0.1], 
              'n_estimators':[150,200,250, 300, 350], 
              'subsample':[ 0.5, 0.7, 0.9, 0.95, 1], 
              'max_features':[0.75, 0.8, 0.85, 0.9, 0.95] }

# Instantiate gb
model_gb = GradientBoostingRegressor()

clf = GridSearchCV(model_gb, parameters, n_jobs=2, verbose=1)

clf.fit(x_train, y_train.values.reshape(-1))

print(clf.best_params_)
print(clf.best_score_)
best_gbr = clf.best_estimator_

Fitting 5 folds for each of 125 candidates, totalling 625 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   17.5s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:  1.6min
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:  3.9min
[Parallel(n_jobs=2)]: Done 625 out of 625 | elapsed:  5.8min finished


{'learning_rate': 0.1, 'max_depth': 4, 'max_features': 0.85, 'n_estimators': 250, 'random_state': 2, 'subsample': 0.95}
0.9873006594789248


In [ ]:
y_regression1 = best_gbr.predict(x_test)

mse = sqrt(mean_squared_error( y_test, y_regression1))

print('MSE: '+str(mse))
print('R2: '+str(best_gbr.score(x_test, y_test)))

MSE: 473.89907573037306
R2: 0.9825684914655469


### Questão 3

Treine um modelo de regressão com o método XGBoost no conjunto de treino. Calcule o Mean Squared Error (MSE) e o R2 no conjunto de teste.

- Dica: Utilize `xgboost.XGBRegressor`

In [ ]:
import xgboost as xgb


xgbr = xgb.XGBRegressor(verbosity=0)

xgbr.fit(x_train, y_train)


y_regression2 = xgbr.predict(x_test)

mse = sqrt(mean_squared_error( y_test, y_regression1))

print('MSE: '+str(mse))
print('R2: '+str(xgbr.score(x_test, y_test)))

MSE: 473.89907573037306
R2: 0.9805709543413166


### Questão 4

Compare seus resultados com os modelos do exercício anterior (DT e RF). Qual se saiu melhor? Por que você acredita que este foi o melhor método?

- Dica: Cuidado para usar o mesmo train/test split em todos os modelos para que a comparação faça sentido. Para isso você pode treinar os modelos da semana passada nesse notebook ou utilizar o mesmo `random_state` na função `train_test_split` nos dois notebooks.




#### Resposta: 

In [ ]:
#Treinando um modelo de regressão com Árvore de Decisão (DT) no conjunto de treino

from sklearn import tree

modeloDTR = tree.DecisionTreeRegressor(random_state=0)

modeloDTR.fit(x_train, y_train)

y_regression = modeloDTR.predict(x_test)

print('Resultados usando o método de Árvore de Decisão:')

#Calculando o r² no conjunto de teste
r2_test = modeloDTR.score(x_test, y_test)
print('R2 no conjunto de teste é: %.4f' % r2_test)

#Calculando MSE no conjunto de teste
mse = sqrt(mean_squared_error( y_test, y_regression1))
print('MSE no conjunto de teste é: %.4f' % mse)

Resultados usando o método de Árvore de Decisão:
R2 no conjunto de teste é: 0.9749
MSE no conjunto de teste é: 473.8991


In [ ]:
#Treinando modelo de regressão com o Random Forest (RF) no conjunto de treino

from sklearn.ensemble import RandomForestRegressor
import sys

modelRF = RandomForestRegressor(random_state=0)

modelRF.fit(x_train, y_train)

print('Resultados usando o método Random Forest:')

#Calculando R²
r2_test = modelRF.score(x_test, y_test)
print('R2 no conjunto de teste é: %.4f' % r2_test)

y_modelRFession = modelRF.predict(x_test)

#Calculando MSE
mse = sqrt(mean_squared_error( y_test, y_regression1))
print('MSE no conjunto de teste é: %.4f' % mse)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Resultados usando o método Random Forest:
R2 no conjunto de teste é: 0.9778
MSE no conjunto de teste é: 473.8991


**Análise Comparativa:**
Segundo os nossos resultados, as técnicas que obtiveram uma melhor performance com o dataset houses_to_rent_v2.csv foram as técnicas de Boosting. A métrica MSE teve o resultado bem aproximado nas técnicas comparadas. Já o R² obteve melhores resultados nas técnicas de Boosting, chegando a 0.98. Nós acreditamos que os métodos Boosting alcançaram melhores resultado devido ao fato deles terem a capacidade de lidar com os valores ausentes internamente (não precisaram de um tratamento separado para os recursos), enquanto que as técnicas DT e RF precisaram de um tratamento antes de serem treinadas, ou seja, precisaram de um tratamento separado para lidar com os valores ausentes no dataset. Enfim, as técnicas Boosting sabem lidar melhor a seleção de features. 

### Questão 5

Verifique a importância de cada feature para o resultado final utilizando o melhor modelo. O que isso diz sobre o problema que estamos tentando resolver?

In [ ]:

labels_city = set(data['city'])
labels_furniture = set(data['furniture'].head())
label_bathroom = set(data['bathroom'].head())

data.head()


,city,rooms,bathroom,parking spaces,furniture,fire insurance (R$),rent amount (R$)
0,4,2,1,1,0,42,3300
1,4,4,4,0,1,63,4960
2,2,1,1,1,1,41,2800
3,2,2,1,0,1,17,1112
4,4,1,1,0,1,11,800


In [ ]:
#feature mais informativa

for feature,importancia in zip(data.columns,best_gbr.feature_importances_):
    print("{}: {}".format(feature, importancia))

city: 0.004645987264516276
rooms: 0.0010829178747352335
bathroom: 0.07309731648251537
parking spaces: 0.0013196179024424657
furniture: 0.0004137162058250249
fire insurance (R$): 0.9194404442699656


Segundo os nossos testes o melhor modelo foi usando o método **Gradient Boosting**. Notamos que a cidade de São Paulo possui em média alugueis mais caros entre os locais analisados, os valores vão entre 800 a 
4.960 reais. Além disso, as Feature relacionadas a mobília (Furniture), quantidade de quartos (bathroom) e número de vagas de estacionamento (parking spaces) possuem uma certa influência no valor final do aluguel. Porém, os nossos testes detectaram que a feature mais importante e que mais influencia no valor final do aluguel é a taxa de incêndio (fire insurance R$). 